In [5]:
import numpy as np
import pandas as pd
import xarray as xr
from datetime import datetime, timedelta
from tobac_flow.flow import Flow
from tobac_flow import io, abi

import cv2 as cv
from scipy import ndimage as ndi


In [2]:
goes_data_path = '../data/GOES16'
start_date = datetime(2018,6,19,17)
days = timedelta(days=0.16)
dates = pd.date_range(start_date, start_date+days, freq='H', closed='left').to_pydatetime()

abi_files = io.find_abi_files(dates, satellite=16, product='MCMIP', view='C', mode=3, 
                              save_dir=goes_data_path, 
                              replicate_path=True, check_download=True, 
                              n_attempts=1, download_missing=True)
             

abi_dates = [io.get_goes_date(i) for i in abi_files]
print(len(abi_files))

# Get time difference (in seconds) between each file
dt = [(abi_dates[1]-abi_dates[0]).total_seconds()/60] \
     + [(abi_dates[i+2]-abi_dates[i]).total_seconds()/120 \
        for i in range(len(abi_files)-2)] \
     + [(abi_dates[-1]-abi_dates[-2]).total_seconds()/60]
dt = np.array(dt)

# Test with some multichannel data
ds_slice = {'x':slice(1325,1475), 'y':slice(725,850)}
# Load a stack of goes datasets using xarray. Select a region over Northern Florida. (full file size in 1500x2500 pixels)
goes_ds = xr.open_mfdataset(abi_files, concat_dim='t', combine='nested').isel(ds_slice)

48


In [3]:
# Extract fields and load into memory
wvd = goes_ds.CMI_C08 - goes_ds.CMI_C10
try:
    wvd = wvd.compute()
except AttributeError:
    pass

bt = goes_ds.CMI_C13
try:
    bt = bt.compute()
except AttributeError:
    pass

swd = goes_ds.CMI_C13 - goes_ds.CMI_C15
try:
    swd = swd.compute()
except AttributeError:
    pass

In [6]:
flow_kwargs = {'pyr_scale':0.5, 'levels':5, 'winsize':16, 'iterations':3, 
               'poly_n':5, 'poly_sigma':1.1, 'flags':cv.OPTFLOW_FARNEBACK_GAUSSIAN}

flow = Flow(bt, flow_kwargs=flow_kwargs, smoothing_passes=3)

In [7]:
%load_ext memory_profiler

In [8]:
%memit flow.sobel(wvd, direction='uphill')


peak memory: 752.70 MiB, increment: 32.07 MiB


In [22]:
%timeit flow.sobel(wvd, direction='uphill')


888 ms ± 49.7 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [11]:
%memit flow.sobel(wvd, direction='uphill', method='nearest')


peak memory: 199.18 MiB, increment: 39.55 MiB


In [23]:
%timeit flow.sobel(wvd, direction='uphill', method='nearest')


806 ms ± 45.9 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [12]:
from tobac_flow.legacy_flow import Flow_Func, flow_sobel, flow_convolve_nearest

In [10]:
l_flow = Flow_Func(flow.flow_for[...,0], flow.flow_back[...,0],
                              flow.flow_for[...,1], flow.flow_back[...,1])

In [20]:
def legacy_sobel(flow, field):
    l_flow = Flow_Func(flow.flow_for[...,0], flow.flow_back[...,0],
                              flow.flow_for[...,1], flow.flow_back[...,1])
    
    def _sobel_func_uphill(x, axis=0):
        sobel_matrix = flow._sobel_matrix(3)
        x = np.fmax(x-x[13],0)
        out_array = np.nansum(x * sobel_matrix.ravel()[:,np.newaxis,np.newaxis], 0)**2
        out_array += np.nansum(x * sobel_matrix.transpose([1,2,0]).ravel()[:,np.newaxis,np.newaxis], 0)**2
        out_array += np.nansum(x * sobel_matrix.transpose([2,0,1]).ravel()[:,np.newaxis,np.newaxis], 0)**2

        return out_array ** 0.5
    
    result = flow_convolve_nearest(field.data, l_flow, structure=np.ones([3,3,3]), function=_sobel_func_uphill)
    
    return result

In [21]:
%memit legacy_sobel(flow, wvd)

peak memory: 251.73 MiB, increment: 9.80 MiB


In [24]:
%timeit legacy_sobel(flow, wvd)

2.34 s ± 19.1 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
